In [3]:
# ! pip install pandas 
# ! pip install numpy
# ! pip install gensim.models

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim import models
from gensim.models import KeyedVectors
import ast
import re
import pickle

# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 유사 태그 출력
from heapq import nlargest
from heapq import nsmallest

# # 모델에 없는 단어들 형태소 분석 
# from konlpy.tag import Okt
# from konlpy.utils import pprint
# okt = Okt()

#### DATA

In [2]:
# 데이터 받아오기 
chart_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/chart.pickle")
genre_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/genre.pickle")
playlist_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist.pickle")
id = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist_id.pkl")

model_path = "C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Model/cc.ko.300.vec"

In [3]:
# 전처리
playlist_tag.rename(columns = {'발매일':'발매년도', '좋아요':'좋아요 수'}, inplace = True)

chart_tag2 = chart_tag.drop('연도', axis = 1)
playlist_tag2 = playlist_tag.drop('플리', axis = 1)
songs_tag = pd.concat([chart_tag2, genre_tag, playlist_tag2])

In [4]:
# 합치기! 

songs_tag = songs_tag[songs_tag['발매년도'] != '-']
songs_tag.loc[:, '발매년도'] = songs_tag['발매년도'].apply(lambda x: str(x)[:4] if len(str(x)) > 4 else str(x))
songs_tag.loc[:, '좋아요 수'] = songs_tag['좋아요 수'].apply(lambda x: int(str(x).replace(',','')))
songs_tag = songs_tag.astype({'발매년도':'int32', '좋아요 수':'int32'})
songs_tag = songs_tag.drop_duplicates(['제목','가수'], keep = 'first')

In [5]:
# 태그 전처리 
hashtags = id.copy()

# 해시태그 없는 플레이리스트 제외
hashtags = hashtags[hashtags['tag'].str.contains("#")]
# 리스트로 전부 가져오기
only_tags = hashtags['tag'].tolist()
# "#" 제거하기
tags_fin  = []
for tag in only_tags:
  tags_fin.append(re.sub(r"[#\[\]\'\,]","",tag).split())
tags_fin[:2]

[['여름', '더워', '시원', '청량', '플리', '케이팝', 'kpop', '키오프'],
 ['팝송', '케이팝', '매장', '카페', '식당', '브금', '노래', '휴식', '신나는', '일']]

트랙 1) 일기 태그와 유사한 플리 찾아서 출력

트랙 2) 일기 태그와 가사 태그가 유사한 노래들

음악 취향 입력
일기 태그, 가사 태그 각각

#### 시도 1 - word2vec, OOV 삭제 

##### 트랙 1) 플리 태그 기반 플리 찾기

In [6]:
# 모델 학습 
word2vec_playlist = Word2Vec(sentences = tags_fin, vector_size = 50, window = 3, min_count = 1, sg = 1, workers = 4)

# 각 단어와 비슷한 단어 찾기... 성능 체크
print("이별이랑 비슷:", word2vec_playlist.wv.most_similar("이별"))
print("사랑이랑 비슷:", word2vec_playlist.wv.most_similar("사랑"))
print("여름이랑 비슷:", word2vec_playlist.wv.most_similar("여름"))

이별이랑 비슷: [('추억', 0.9958157539367676), ('우울', 0.9957407712936401), ('야경', 0.9955396056175232), ('잔잔한', 0.9953756332397461), ('새벽', 0.995330274105072), ('카페', 0.9953253865242004), ('인디', 0.9951081871986389), ('세련된', 0.9950740933418274), ('센치한', 0.9950241446495056), ('감성힙합', 0.9949735999107361)]
사랑이랑 비슷: [('세련된', 0.9960087537765503), ('발라드', 0.995893657207489), ('감성', 0.995736300945282), ('센치한', 0.9956501722335815), ('겨울', 0.9954304099082947), ('행복', 0.9953859448432922), ('편안한', 0.9953819513320923), ('우울', 0.9953790307044983), ('한강', 0.9953010678291321), ('노래방', 0.9952926635742188)]
여름이랑 비슷: [('발라드', 0.9968786239624023), ('청춘', 0.9961419105529785), ('겨울', 0.9959815740585327), ('피크닉', 0.9959700107574463), ('인디', 0.9958917498588562), ('Pop', 0.9958612322807312), ('케이팝', 0.9958339929580688), ('트렌디', 0.9958193302154541), ('설레는', 0.9956774711608887), ('세련된', 0.9956440925598145)]


###### 성능이 엄청 막 좋아보이지는 않...아

In [7]:
# 기존 태그 임베딩, 벡터 평균 함수 
def vector_and_average(input_lists, model):
  avg_vectors = []
  for list in input_lists:
    vectors = [model.wv[word] for word in list]
    avg = sum(vectors) / len(vectors)
    avg_vectors.append(avg)
  return avg_vectors

# 새로운 입력 - 모델에 있는 단어들만을 가지고 평균 벡터 생성
def new_tag_vector(tag, model):
  tag_vector = []
  for word in tag:
    if word in model.wv:
      tag_vector.append(model.wv[word])
    else:
      print(f"{word} 단어는 모델에 없습니다..")

  avg_vector = sum(tag_vector)/len(tag_vector)
  return avg_vector

# 코사인 유사도 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 유사 해시태그 출력 함수
def similar_things(new_input, original_tags, model, num):
  new_vector = new_tag_vector(new_input, model)
  original_vectors = vector_and_average(original_tags, model)
  # 유사도 계산
  similarities = []
  for i in range(len(original_tags)):
    similarities.append(cos_sim(new_vector, original_vectors[i]))
  # 유사 해시태그 출력
  top_index = [idx for idx,_ in nlargest(num, enumerate(similarities), key = lambda x:x[1])] # enumerate 에서 두 번째 요소(유사도) 기준
  top_tags = [original_tags[idx] for idx in top_index] # 유사한 "태그"
  # print(top_index) # 유사한 "플리 index"
  return top_index, top_tags

In [8]:
# 기존 해시태그들 벡터화, 평균 계산
tags_vector = []
for tag in tags_fin:
  tags_vector.append(new_tag_vector(tag, word2vec_playlist))

print(f"첫 해시태그의 벡터화 : \n {tags_vector[0]}")

# 새로운 태그에 대한 평균 벡터 생성
new_tag = ['겨울', '사랑', '눈', '몽글몽글몽글몽글']
new_vector = new_tag_vector(new_tag, word2vec_playlist)

# 새로운 input과 유사한 애들
sim = similar_things(['신남', '댄스', '드라이브', '여름', '더워', '흥분'], tags_fin, word2vec_playlist, 5)
sim[0]

첫 해시태그의 벡터화 : 
 [ 0.04174288 -0.07175546  0.03454174  0.03228088 -0.03121519 -0.15712497
  0.15373649  0.20935765 -0.22676413 -0.11801913  0.02619883 -0.1937085
  0.15632737  0.0937292  -0.03358329  0.11670556  0.06203141  0.01118164
 -0.25351626 -0.10508794  0.01724208  0.14439414  0.24781309 -0.06193255
  0.15933226  0.00472437 -0.10877293  0.05716702 -0.16969669  0.0780718
  0.01862974 -0.07770737 -0.01380573 -0.06982583 -0.15681936  0.1373198
  0.12205312 -0.02337974  0.02634025 -0.01653233  0.19973798 -0.01456144
 -0.01904519 -0.00472036  0.31496435  0.01789968 -0.11823456 -0.0732379
  0.07841399  0.13612808]
몽글몽글몽글몽글 단어는 모델에 없습니다..
흥분 단어는 모델에 없습니다..


[301, 728, 161, 813, 816]

In [9]:
# 추천 플레이리스트와 해시태그 
id.iloc[sim[0]]

,name,tag,id
301,output_17,"['#여름', '#신나는', '#팝', '#댄스', '#하우스', '#드라이브']",302
728,ozu_4,"['#인디', '#위로', '#힘들때', '#우울', '#지친일상', '#하루의끝']",729
161,moonsound_14,"['#여름', '#남자아이돌', '#댄스', '#드라이브', '#신나는', '#기분...",162
813,shorthair_18,"['#감성', '#추억', '#지하철', '#가사', '#보고싶다', '#사랑', ...",814
816,jjinmak_1,"['#응원', '#기분전환', '#위로', '#희망', '#힐링', '#휴식', '...",817


##### 트랙 2) 음악 취향 필터링, 가사 태그 기반 노래 찾기 

In [10]:
# 결측치 제거 
songs_tag_nona = songs_tag[~songs_tag['키워드'].isna()].copy()

# 키워드 + 감정 
songs_tag_nona['tags'] = songs_tag_nona['키워드'] + songs_tag_nona['감정']

songs_tags = songs_tag_nona['tags'].tolist()

In [11]:
# 모델 학습 
word2vec_songs = Word2Vec(sentences = songs_tags, vector_size = 100, window = 4, min_count = 1, workers = 4)

print("이별이랑 비슷:", word2vec_songs.wv.most_similar("이별"))
print("사랑이랑 비슷:", word2vec_songs.wv.most_similar("사랑"))
print("여름이랑 비슷:", word2vec_songs.wv.most_similar("여름"))

이별이랑 비슷: [('미련', 0.9965863823890686), ('눈물', 0.996432900428772), ('헤어짐', 0.9945165514945984), ('허전함', 0.9924057126045227), ('후회', 0.9920188188552856), ('아픔', 0.9888947606086731), ('미안함', 0.9880008697509766), ('기억', 0.9863479733467102), ('비', 0.9846117496490479), ('슬픔', 0.9840127229690552)]
사랑이랑 비슷: [('우리', 0.9885333776473999), ('당신', 0.9868236780166626), ('고백', 0.9836774468421936), ('마음', 0.982589840888977), ('영원', 0.9825274348258972), ('기다림', 0.9824977517127991), ('약속', 0.9803306460380554), ('눈', 0.978808581829071), ('미소', 0.9772049188613892), ('봄', 0.9770039916038513)]
여름이랑 비슷: [('바다', 0.991234540939331), ('감동', 0.990029513835907), ('별', 0.9897505044937134), ('따뜻함', 0.9886747002601624), ('순간', 0.988482654094696), ('크리스마스', 0.9882587194442749), ('안락함', 0.9848021864891052), ('설레임', 0.9840795993804932), ('하늘', 0.9839812517166138), ('안정', 0.98306804895401)]


In [12]:
# 기존 태그 벡터화 
original_vectors = vector_and_average(songs_tags, word2vec_songs)

print(len(original_vectors)) 

# 새로운 태그 벡터화 
idx, tag = similar_things(new_tag, songs_tags, word2vec_songs, 2)
songs_tag_nona.iloc[idx,:]

24088
몽글몽글몽글몽글 단어는 모델에 없습니다..


,제목,가수,가사,발매년도,장르,좋아요 수,키워드,감정,tags
6335,하나뿐인 사람,케이윌,그대의 눈을 보면 떨려오는 가슴에 설레곤해 그대를 바라보면 온 세상이 멈춰버린듯 느...,2014,"발라드, 국내드라마",19005,"[그대, 가슴, 봄]","[설레임, 믿음, 사랑]","[그대, 가슴, 봄, 설레임, 믿음, 사랑]"
2548,Mine (POP Mix),Taylor Swift,You were in college workin' part time waitin' ...,2010,포크/블루스/컨트리,1990,"[물가, 파트타임, 사랑]","[추억, 감동, 두려움]","[물가, 파트타임, 사랑, 추억, 감동, 두려움]"


### 최종 !!!

In [14]:
#### 일기 태그 ####

# ! pip install openai
# ! pip install python-dotenv

import os
from openai import OpenAI
from dotenv import load_dotenv
import ast

# API 
def get_api():
    load_dotenv()
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    return OPENAI_API_KEY

api = get_api()
client = OpenAI(api_key = api)

# Keywords
def get_keywords(input_text):

    prompt = f"""
    You are a helpful asssistant. 
    Extract **four emotions** and **four keyword** in **Korean** from the following text : {input_text}.
    Make sure all the answers are **nouns**.
    Follow the given format : 
    {["emotion1", "emotion2", "emotion3", "emotion4", "keyword1", "keyword2", "keyword3", "keyword4"]}
    """

    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        # max_tokens = 50,
        # temperature = 0.3
    )

    keywords = response.choices[0].message.content
    return keywords 

####################################################################################
input_text = """
방학이 벌써 반이나 지나가버렸어. 
오랫동안 기대하고 꿈꿨던 방학이었는데.. 벌써 반이나 갔다니 너무 아쉬워ㅠㅠ
그래도 남은 절반을 신나고 행복하게 즐겨볼 테야!
이제 연휴니까 푹 쉬고 열심히 공부도 해야지!
"""
#####################################################################################

diary_keywords = get_keywords(input_text)

diary_tags = ast.literal_eval(diary_keywords)
print(diary_tags)

['기대', '꿈', '아쉬움', '신남', '방학', '절반', '행복', '공부']


#### 시도 1

In [15]:
# 트랙 1

# 일기 태그와 모델 
filtered_diary_tags = [word for word in diary_tags if word in word2vec_playlist.wv]

# 유사한 플레이리스트 
idx, _ = similar_things(filtered_diary_tags, tags_fin, word2vec_playlist, 5)
id.iloc[idx, [0,1]]

,name,tag
728,ozu_4,"['#인디', '#위로', '#힘들때', '#우울', '#지친일상', '#하루의끝']"
964,choco_9,"['#청춘', '#일본밴드', '#락', '#청량', '#드라이브', '#스트레스'..."
446,sonik_2,"['#여름밤', '#열대야', '#팝송', '#감성', '#밤', '#새벽', '#..."
101,mogu_4,"['#Rock', '#감성', '#록', '#밴드', '#기분전환', '#휴식', ..."
772,miles_8,"['#중세시대', '#케이팝', '#수록곡', '#아이돌', '#화려한', '#무도..."


In [17]:
# 트랙 2

# 일기 태그와 모델 
filtered_diary_tags = [word for word in diary_tags if word in word2vec_songs.wv]

# 사용자 입력
genre_input = input("어떤 장르의 음악을 좋아하시나요? (예: 힙합, 발라드 등): ")
likes_input = input("유명한 곡들이 좋으신가요? (o, x로 답해주세요!): ")
release_input = input("최신 노래를 좋아하시나요? (o, x로 답해주세요!): ")

# 취향 필터링 
def filtering(genre_input, release_input, likes_input):
    # 장르 필터링 
    genre_df = songs_tag_nona[songs_tag_nona['장르'].str.contains(genre_input)]

    # 발매년도 필터링 
    if release_input == 'o':
        year_df = genre_df[genre_df['발매년도'] >= 2010]
    elif release_input == 'x':
        year_df = genre_df[genre_df['발매년도'] < 2010]
    else:
        print("o 또는 x로 답변해주세요 :)")

    upper_60 = year_df['좋아요 수'].quantile(0.6) # 60% 지점 
    lower_40 = year_df['좋아요 수'].quantile(0.4) # 40% 지점

    if likes_input == 'o':
        likes_df = year_df[year_df['좋아요 수'] >= upper_60]
    elif likes_input == 'x':
        likes_df = year_df[year_df['좋아요 수'] <= lower_40]
    else:
        print("o 또는 x로 답변해주세요 :)")
    
    if likes_df.empty:
        print("취향에 맞는 곡이 없습니다ㅠㅠ")
    else: 
        sim_indices, sim_tags = similar_things(filtered_diary_tags, likes_df['tags'].tolist(), word2vec_songs, 15)
        return likes_df.iloc[sim_indices,[0,1,8]]

print(filtering(genre_input, likes_input, release_input))


                                          제목                     가수   
5615                            Come With Me  Surfaces, salem ilese  \
21647                     The Christmas Song         Johnny Stimson   
12530               if this is the last time                   LANY   
5222                       떠나지 못할 이유는 없었다는 걸                     하림   
3652                      Summer In New York            Sofi Tukker   
29374                              Moonlight  AB6IX (에이비식스), Reiley   
1582                                Cat Cafe                 Shoffy   
12352                                 sunday              hard life   
12114                        You Got Me Like            SHAED, snny   
18350         もう少しだけ (조금만 더, Mousukoshidake)                YOASOBI   
11353                        Straight To You       Stephanie Poetri   
12409                                   Easy            Peder Elias   
7108                                 Skyline                 Khalid   
28617 